<a href="https://colab.research.google.com/github/haariswaqas/Group_7/blob/main/AI_LAB3_SUDOKU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TASK 1: Problem Formulation**


## **1.**

* ###    **Iniital State:** A partially filled 9x9 Sudoko puzzle where the empty cells are filled with 0

* ###    **Successor Function:** The successor function generates a new state by assigning a number (1-9) to a blank cell. The function ensures that the assignment does not violate any constraints (row, column, sub-grid).

* ###  **Goal Test:** The goal test checks if the current state is a complete and valid Sudoku solution, i.e., all cells are filled, and all constraints are satisfied.



## **2.**

* ###  **Row Constraint:** ∀i ∈ [1, 9], ∀j, k ∈ [1, 9], i ≠ k, Sij ≠ Sik

* ###  **Column Constraint:** ∀j ∈ [1, 9], ∀i, k ∈ [1, 9], i ≠ k, Sij ≠ Skj

* ### **Sub-grid Constraint:** ∀m, n ∈ [1, 3], ∀i, j ∈ [(m-1)×3 + 1, m×3], [(n-1)×3 + 1, n×3], (i ≠ k ∨ j ≠ l) ⇒ Sij ≠ Skl

## Where:

### - Sij represents the cell at row i and column j
### - i, j, k, l are indices representing rows and columns
### - m, n are indices representing sub-grid rows and columns





# **TASK 2: Sudoku Solver using Backtracking**

This code implements a Sudoku solver using the backtracking algorithm. It consists of three functions:

* ### `is_valid`: Checks if a given number can be placed in a specified cell according to Sudoku rules.
* ### `solve_sudoku`: Uses backtracking to solve the Sudoku puzzle by iteratively filling in empty cells with valid numbers.
* ### `print_board`: Prints the Sudoku board in a user-readable format.

### The solver takes a 9x9 array representing the Sudoku board as input and returns the solved board.

In [17]:
import time
import sys

def is_valid(board, row, col, num):
    # Check the row
    for x in range(9):
        if board[row][x] == num:
            return False

    # Check the column
    for x in range(9):
        if board[x][col] == num:
            return False

    # Check the box
    start_row = row - row % 3
    start_col = col - col % 3
    for i in range(3):
        for j in range(3):
            if board[i + start_row][j + start_col] == num:
                return False
    return True


def solve_sudoku(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                for num in range(1, 10):
                    if is_valid(board, i, j, num):
                        board[i][j] = num
                        if solve_sudoku(board):
                            return True
                        board[i][j] = 0
                return False
    return True


def print_board(board):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - -")
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("| ", end="")
            if j == 8:
                print(board[i][j])
            else:
                print(str(board[i][j]) + " ", end="")


# Test the code
board = [
    [4, 0, 6, 5, 0, 2, 8, 0, 9],
    [0, 0, 0, 0, 4, 0, 0, 3, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 5],
    [6, 0, 0, 8, 0, 0, 1, 0, 0],
    [5, 0, 0, 0, 7, 0, 0, 8, 0],
    [3, 0, 2, 9, 0, 4, 0, 6, 0],
    [0, 2, 0, 6, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 5, 3, 9, 4, 0],
    [8, 3, 0, 0, 9, 0, 0, 7, 2]
]

start_time = time.time()
if solve_sudoku(board):
    print_board(board)
else:
    print("No solution exists")

print("Time taken:", time.time() - start_time)
print("Memory usage:", sys.getsizeof(board))

4 7 6 | 5 3 2 | 8 1 9
2 5 8 | 1 4 9 | 7 3 6
1 9 3 | 7 6 8 | 4 2 5
- - - - - - - - - - - -
6 4 7 | 8 2 5 | 1 9 3
5 1 9 | 3 7 6 | 2 8 4
3 8 2 | 9 1 4 | 5 6 7
- - - - - - - - - - - -
9 2 4 | 6 8 7 | 3 5 1
7 6 1 | 2 5 3 | 9 4 8
8 3 5 | 4 9 1 | 6 7 2
Time taken: 0.008668899536132812
Memory usage: 128


# **Task 3: Backtracking with Forward Checking**


In this task, we extend the Backtracking Search algorithm to incorporate Forward Checking as a constraint propagation technique. Forward Checking reduces the search space by eliminating values from the domain of unassigned variables that are inconsistent with the current assignment.

We implement the following functions:

* `initialize_domains`: Initializes the domain of possible values for each cell.
* `forward_checking`: Updates the domain of each unassigned variable when a number is placed in a cell.
* `mrv`: Selects the next cell to fill using the Minimum Remaining Values (MRV) heuristic.
* `solve_sudoku`: Solves the Sudoku puzzle using Backtracking with Forward Checking.

In [19]:

import timeit
import sys

# Function to measure memory usage
def get_memory_usage(obj):
    return sys.getsizeof(obj)

# Function to initialize the domain of possible values for each cell
def initialize_domains(board):
    domains = {}
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                domains[(row, col)] = {num for num in range(1, 10) if is_valid(board, row, col, num)}
            else:
                domains[(row, col)] = set()
    return domains

# Function to check whether placing a given number in a specified cell is valid according to Sudoku rules
def is_valid(board, row, col, num):
    # Check the row
    if num in board[row]:
        return False

    # Check the column
    for i in range(9):
        if board[i][col] == num:
            return False

    # Check the 3x3 sub-grid
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(start_row, start_row + 3):
        for j in range(start_col, start_col + 3):
            if board[i][j] == num:
                return False

    return True

# Function to perform forward checking
def forward_checking(domains, row, col, num):
    affected_cells = []
    for i in range(9):
        if (row, i) in domains and num in domains[(row, i)]:
            domains[(row, i)].remove(num)
            affected_cells.append((row, i))
        if (i, col) in domains and num in domains[(i, col)]:
            domains[(i, col)].remove(num)
            affected_cells.append((i, col))

    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(start_row, start_row + 3):
        for j in range(start_col, start_col + 3):
            if (i, j) in domains and num in domains[(i, j)]:
                domains[(i, j)].remove(num)
                affected_cells.append((i, j))

    return affected_cells

def undo_forward_checking(domains, affected_cells, num):
    for (row, col) in affected_cells:
        domains[(row, col)].add(num)

# Function to select the next cell to fill using the Minimum Remaining Values (MRV) heuristic
def mrv(domains):
    min_domain_size = 10
    best_cell = None
    for cell, domain in domains.items():
        if len(domain) > 0 and len(domain) < min_domain_size:
            min_domain_size = len(domain)
            best_cell = cell
    return best_cell

# Function to solve the Sudoku puzzle using backtracking with forward checking
def solve_sudoku(board):
    domains = initialize_domains(board)
    if backtrack(board, domains):
        return board, domains
    else:
        return None, domains


def backtrack(board, domains):
    cell = mrv(domains)
    if not cell:
        return True  # All cells filled

    row, col = cell
    for num in domains[cell]:
        if is_valid(board, row, col, num):
            board[row][col] = num
            affected_cells = forward_checking(domains, row, col, num)
            if all(len(domains[cell]) > 0 for cell in domains if len(domains[cell]) > 0):
                if backtrack(board, domains):
                    return True
            board[row][col] = 0  # Backtrack
            undo_forward_checking(domains, affected_cells, num)

    return False

# Function to print the Sudoku board
def print_board(board):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            print(board[i][j], end=" ")
        print()

# Example Sudoku puzzle
board = [
    [4, 0, 6, 5, 0, 2, 8, 0, 9],
    [0, 0, 0, 0, 4, 0, 0, 3, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 5],
    [6, 0, 0, 8, 0, 0, 1, 0, 0],
    [5, 0, 0, 0, 7, 0, 0, 8, 0],
    [3, 0, 2, 9, 0, 4, 0, 6, 0],
    [0, 2, 0, 6, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 5, 3, 9, 4, 0],
    [8, 3, 0, 0, 9, 0, 0, 7, 2]
]

# Measure time and memory usage
start_time = timeit.default_timer()  # Start timer
solved_board, final_domains = solve_sudoku(sudoku_board)  # Solve Sudoku puzzle
end_time = timeit.default_timer()  # End timer

# Calculate time taken
time_taken = end_time - start_time

# Print results
print("Solution using Backtracking with Forward Checking:")
if solved_board:
    print_board(solved_board)
else:
    print("No solution found.")
print("Time taken:", time_taken)
print("Memory usage:", sys.getsizeof(board))

Solution using Backtracking with Forward Checking:
4 7 6 | 5 3 2 | 8 1 9 
2 5 8 | 1 4 9 | 7 3 6 
1 9 3 | 7 6 8 | 4 2 5 
---------------------
6 4 7 | 8 2 5 | 1 9 3 
5 1 9 | 3 7 6 | 2 8 4 
3 8 2 | 9 1 4 | 5 6 7 
---------------------
9 2 4 | 6 8 7 | 3 5 1 
7 6 1 | 2 5 3 | 9 4 8 
8 3 5 | 4 9 1 | 6 7 2 
Time taken: 0.0001684490000570804
Memory usage: 128


# **Performance Comparison**

### Task 2: Backtracking Search

* **Time taken: 0.008668899536132812**
* **Memory usage: 128**

### Task 3: Backtracking with Forward Checking

* **Time taken: 0.0001684490000570804**
* **Memory usage: 128**




The results show that the Backtracking with Forward Checking algorithm (Task 3) outperforms the Backtracking Search algorithm (Task 2) in terms of time taken. The Forward Checking algorithm reduces the search space by eliminating inconsistent values, resulting in a significant speedup.

On the other hand, both algorithms have the same memory usage, which is expected since they use the same data structures to represent the Sudoku board.

The performance metrics can be attributed to the following reasons:

* ### Backtracking Search (Task 2):
    * The algorithm explores the entire search space, resulting in a higher time complexity.
    * The algorithm does not use any constraint propagation techniques, leading to a larger search space.
* ### Backtracking with Forward Checking (Task 3):
    * The algorithm uses Forward Checking to reduce the search space, resulting in a lower time complexity.
    * The algorithm uses the Minimum Remaining Values (MRV) heuristic to select the next cell to fill, which further reduces the search space.

In conclusion, the Backtracking with Forward Checking algorithm is more efficient than the Backtracking Search algorithm due to its use of constraint propagation and heuristic search.